In [ ]:
#| default_exp logger

In [ ]:
#| export

from typing import *
import logging

In [ ]:
#| exporti

import logging.config
from typer import echo

In [ ]:
#| exporti

logging_level = logging.WARNING

In [ ]:
import pytest

import airt.sanitizer

In [ ]:
#| exporti

# Logger Levels
# CRITICAL = 50
# ERROR = 40
# WARNING = 30
# INFO = 20
# DEBUG = 10
# NOTSET = 0

# Create a custom logger and set the configuration to echo the logging messages
# Reference: https://github.com/tiangolo/typer/issues/203


class CLILoggerHandler(logging.Handler):
    """A custom logger handler class that use Typer echo to log results."""

    def emit(self, record: logging.LogRecord) -> None:
        """A function to channel logger outout to Typer echo."""
        
        echo(self.format(record))
        
        
# Global handler for the loggers
handler = CLILoggerHandler()


In [ ]:
#| export

should_supress_timestamps: bool = False
    
def supress_timestamps(flag:bool = True):
    global should_supress_timestamps
    should_supress_timestamps = flag

def get_logger(name: str) -> logging.Logger:
    """An internal function to set the default logger configuration

    Returns:
        An instance of Logger with custom configuration that can be used across the CLI for logging
    """
    
    # Logger Configuration
    
    global should_supress_timestamps
    
    if should_supress_timestamps:
        fmt = "[%(levelname)s] %(name)s: %(message)s"
    else:
        fmt = "%(asctime)s.%(msecs)03d [%(levelname)s] %(name)s: %(message)s"
    
    datefmt = "%y-%m-%d %H:%M:%S"

    formatter = logging.Formatter(fmt=fmt, datefmt=datefmt)

    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

    return logger

In [ ]:
#| include: false

assert type(get_logger(__name__)) == logging.Logger

with pytest.raises(TypeError) as e:
    get_logger()
assert "missing 1 required positional argument" in str(e.value)

In [ ]:
#| export

def set_level(level: int):
    
    # Getting all loggers that has either airt or __main__ in the name
    loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict if ("airt" in name) or ("__main__" in name)]

    for logger in loggers:
        logger.setLevel(level)

In [ ]:
#| include: false

logger = get_logger(__name__)

level = logging.WARNING

set_level(level)

# Checking if the logger is to logging.WARNING in dev mode
display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.WARNING

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

30

22-10-21 08:25:48.423 [WARNING] __main__: This is a warning
22-10-21 08:25:48.424 [ERROR] __main__: This is an error


In [ ]:
#| include: false

supress_timestamps()
logger = get_logger(__name__)

# Checking if the logger is set back to logging.INFO in dev mode
display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.INFO

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

20

[INFO] __main__: This is an info
[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error
